### Import Library

In [12]:
! pip install -r C:\Users\MacBook\OneDrive\Dokumen\VS_Code\magnitudr\requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

### Extract Data

In [14]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
response = requests.get(url)
data = response.json()

earthquakes = []
for feature in data["features"]:
    properties = feature["properties"]
    geometry = feature["geometry"]
    earthquakes.append({
        "id": feature["id"],
        "magnitude": properties["mag"],
        "place": properties["place"],
        "time": properties["time"],
        "longitude": geometry["coordinates"][0],
        "latitude": geometry["coordinates"][1],
        "depth": geometry["coordinates"][2]
    })

df = pd.DataFrame(earthquakes)

In [15]:
df.head()

,id,magnitude,place,time,longitude,latitude,depth
0,tx2025ivwt,1.7,"41 km W of Mentone, Texas",1746560084856,-104.0260,31.6380,4.7063
1,ak0255sp6zre,2.7,"37 km W of Tatitlek, Alaska",1746559692908,-147.3655,60.9206,12.4000
2,ak0255sp5wws,1.4,"35 km WNW of Petersville, Alaska",1746559389409,-151.4007,62.6304,10.3000
3,ak0255sp2a03,1.7,"33 km NNE of Chase, Alaska",1746558372835,-149.8538,62.7256,64.8000
4,nn00897286,1.2,"20 km NNE of Yerington, Nevada",1746558173301,-119.0450,39.1439,11.2000


In [16]:
df = df[(df["longitude"].between(95, 141)) & (df["latitude"].between(-11, 6))]

In [17]:
df["depth_category"] = pd.cut(
    df["depth"],
    bins=[0, 70, 300, 700],
    labels=["Shallow", "Intermediate", "Deep"]
)

In [18]:
df["time"] = pd.to_datetime(df["time"], unit="ms")

df["time"] = df["time"].dt.tz_localize("UTC").dt.tz_convert("Asia/Jakarta")

In [19]:
df.head(30)

,id,magnitude,place,time,longitude,latitude,depth,depth_category
39,us7000pxht,4.6,"75 km SSE of Biak, Indonesia",2025-05-06 21:20:11.207000+07:00,136.3595,-1.7991,27.354,Shallow
98,us7000pxg2,4.6,"196 km SW of Abepura, Indonesia",2025-05-06 14:13:12.258000+07:00,139.2705,-3.7314,56.973,Shallow
144,us7000pxem,4.7,"85 km W of Tobelo, Indonesia",2025-05-06 08:45:48.108000+07:00,127.2424,1.6887,123.931,Intermediate
185,us7000pxdu,4.9,"93 km SW of Modisi, Indonesia",2025-05-06 05:57:33.219000+07:00,123.7543,-0.0463,120.847,Intermediate
333,us7000px7k,4.6,"183 km W of Tual, Indonesia",2025-05-05 15:36:59.235000+07:00,131.1014,-5.5133,69.453,Shallow
410,us7000px5i,4.4,"137 km SE of Sanana, Indonesia",2025-05-05 05:39:40.780000+07:00,126.7938,-2.9968,10.000,Shallow
463,us7000px3z,4.3,"150 km WNW of Tobelo, Indonesia",2025-05-04 23:00:03.968000+07:00,126.7552,2.2282,56.952,Shallow
534,us7000px1y,4.6,"236 km NNW of Tobelo, Indonesia",2025-05-04 15:33:23.775000+07:00,127.1590,3.6838,49.380,Shallow
651,us7000pwyv,4.5,"63 km NNE of Pototano, Indonesia",2025-05-04 04:26:26.077000+07:00,117.7102,-7.8887,10.000,Shallow
654,us7000pwys,4.4,Banda Sea,2025-05-04 04:13:47.179000+07:00,129.9592,-6.8264,120.086,Intermediate


In [22]:
load_dotenv()

engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

In [23]:
df.to_sql("earthquakes", engine, if_exists="append", index=False)

121

In [24]:
df_all = pd.read_sql("SELECT * FROM earthquakes", engine)
df_all.head(30)

,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pxht,4.6,"75 km SSE of Biak, Indonesia",2025-05-06 14:20:11.207000+00:00,136.3595,-1.7991,27.354,Shallow
1,us7000pxg2,4.6,"196 km SW of Abepura, Indonesia",2025-05-06 07:13:12.258000+00:00,139.2705,-3.7314,56.973,Shallow
2,us7000pxem,4.7,"85 km W of Tobelo, Indonesia",2025-05-06 01:45:48.108000+00:00,127.2424,1.6887,123.931,Intermediate
3,us7000pxdu,4.9,"93 km SW of Modisi, Indonesia",2025-05-05 22:57:33.219000+00:00,123.7543,-0.0463,120.847,Intermediate
4,us7000px7k,4.6,"183 km W of Tual, Indonesia",2025-05-05 08:36:59.235000+00:00,131.1014,-5.5133,69.453,Shallow
5,us7000px5i,4.4,"137 km SE of Sanana, Indonesia",2025-05-04 22:39:40.780000+00:00,126.7938,-2.9968,10.000,Shallow
6,us7000px3z,4.3,"150 km WNW of Tobelo, Indonesia",2025-05-04 16:00:03.968000+00:00,126.7552,2.2282,56.952,Shallow
7,us7000px1y,4.6,"236 km NNW of Tobelo, Indonesia",2025-05-04 08:33:23.775000+00:00,127.1590,3.6838,49.380,Shallow
8,us7000pwyv,4.5,"63 km NNE of Pototano, Indonesia",2025-05-03 21:26:26.077000+00:00,117.7102,-7.8887,10.000,Shallow
9,us7000pwys,4.4,Banda Sea,2025-05-03 21:13:47.179000+00:00,129.9592,-6.8264,120.086,Intermediate
